# Read input files

In [1]:
%%time
import pandas as pd

pad = "/kaggle/input/makeparquet"
transactions = pd.read_parquet(pad+'/transactions_train.parquet')
customers = pd.read_parquet(pad+'/customers.parquet')
articles = pd.read_parquet(pad+'/articles.parquet')


CPU times: user 4.4 s, sys: 5.48 s, total: 9.88 s
Wall time: 8.45 s


# Research Question 1: Repurchase Analysis


In [2]:
# Calculate repurchase rate for each article
# Group by customer and article, and count how many times each article was purchased
repurchase_counts = transactions.groupby(['customer_id', 'article_id'])['t_dat'].count().reset_index()
repurchase_counts['t_dat'] = repurchase_counts['t_dat'] - 1

# Calculate the average repurchase rate
average_repurchase_rate = repurchase_counts['t_dat'].mean()
print(average_repurchase_rate)

0.16413289920373725


Bulk purchases:

In [3]:
average_repurchase_rate = repurchase_counts['t_dat'].max()
print(average_repurchase_rate)

569


In [4]:
repurchase_counts[repurchase_counts['t_dat']==569]

,customer_id,article_id,t_dat
8524309,5743206735920807081,678342001,569


In [5]:
articles[articles.article_id == 678342001]

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,fabric_wool,fabric_polyester,fabric_silk,fabric_denim,fabric_linen,fabric_spandex,fabric_rayon,fabric_nylon,fabric_leather,fabric_suede
43839,678342001,678342,Lima SS.,255,T-shirt,Garment Upper body,1010016,Solid,9,Black,...,0,0,0,0,0,0,0,0,0,0


In [6]:
transactions[(transactions.customer_id == 5743206735920807081) & (transactions.article_id == 678342001)]


,t_dat,customer_id,article_id,price,sales_channel_id,season,year,month,week
1177871,2018-10-14,5743206735920807081,678342001,0.006763,1,3,-30,10,3
1177872,2018-10-14,5743206735920807081,678342001,0.006763,1,3,-30,10,3
1177873,2018-10-14,5743206735920807081,678342001,0.006763,1,3,-30,10,3
1177874,2018-10-14,5743206735920807081,678342001,0.006763,1,3,-30,10,3
1177875,2018-10-14,5743206735920807081,678342001,0.006763,1,3,-30,10,3
...,...,...,...,...,...,...,...,...,...
1178436,2018-10-14,5743206735920807081,678342001,0.006763,1,3,-30,10,3
1178437,2018-10-14,5743206735920807081,678342001,0.006763,1,3,-30,10,3
1178438,2018-10-14,5743206735920807081,678342001,0.006763,1,3,-30,10,3
1178439,2018-10-14,5743206735920807081,678342001,0.006763,1,3,-30,10,3


In [7]:
# Calculate the average time between purchases of the same item
transactions['t_dat'] = pd.to_datetime(transactions['t_dat'])
transactions = transactions.sort_values(by=['customer_id', 'article_id', 't_dat'])

# Calculate time difference between successive purchases of the same item
transactions['time_diff'] = transactions.groupby(['customer_id', 'article_id'])['t_dat'].diff()

# Calculate the average time difference for each article
average_time_between_purchases = transactions.groupby('article_id')['time_diff'].mean()
print(average_time_between_purchases.mean())


5 days 09:23:10.299972753


In [8]:
# Calculate repurchase rate for each customer
# Calculate the number of unique items purchased by each customer
unique_items_per_customer = transactions.groupby('customer_id')['article_id'].nunique()

# Calculate the number of repurchased items by each customer
repurchased_items_per_customer = transactions.groupby(['customer_id', 'article_id']).size().groupby('customer_id').sum()

# Calculate the customer-specific repurchase rate
customer_repurchase_rate = (repurchased_items_per_customer / unique_items_per_customer)-1
print(unique_items_per_customer.sum())
print(repurchased_items_per_customer.sum())


print(customer_repurchase_rate.sort_values(ascending=False))

27306439
31788324
customer_id
2729025827381139556     99.0
11558794799858991958    93.0
4485518665254175540     85.0
17539785738842741120    80.0
3601599666106972342     79.0
                        ... 
8168426714032213336      0.0
8168460412275439387      0.0
8168483953316266352      0.0
8168523330546161729      0.0
9234284623732377441      0.0
Length: 1362281, dtype: float64


In [9]:
# Calculate the number of unique customers who purchased each item
unique_customers_per_item = transactions.groupby('article_id')['customer_id'].nunique()

# Calculate the number of repurchases for each item
repurchases_per_item = transactions.groupby(['article_id', 'customer_id']).size().groupby('article_id').sum()

# Calculate the item-specific repurchase rate
item_repurchase = repurchases_per_item - unique_customers_per_item
item_repurchase_rate = repurchases_per_item / (unique_customers_per_item)
filtered_item_repurchase_rate = item_repurchase_rate[item_repurchase_rate != 1]

print(item_repurchase.sort_values(ascending=False))
print(filtered_item_repurchase_rate.sort_values(ascending=False))
print(len(filtered_item_repurchase_rate))
print(len(item_repurchase_rate))

article_id
706016001    18036
156231001    10761
706016002     9558
160442007     7993
610776002     7628
             ...  
528746001        0
423900049        0
423900050        0
556424001        0
565200003        0
Length: 104547, dtype: int64
article_id
740299001    15.000000
704760004     8.000000
331474016     8.000000
877607001     6.333333
701784017     5.750000
               ...    
868629001     1.003831
624634005     1.003731
860885002     1.003623
805510032     1.003236
838750001     1.003205
Length: 81937, dtype: float64
81937
104547


In [10]:
test = transactions[transactions.customer_id ==3601599666106972342]
print(test)

              t_dat          customer_id  article_id     price  \
28675451 2020-07-06  3601599666106972342   685813001  0.016932   
28675452 2020-07-06  3601599666106972342   685813001  0.016932   
28675453 2020-07-06  3601599666106972342   685813001  0.016932   
28675454 2020-07-06  3601599666106972342   685813001  0.016932   
28675455 2020-07-06  3601599666106972342   685813001  0.016932   
...             ...                  ...         ...       ...   
28675526 2020-07-06  3601599666106972342   685813001  0.016932   
28675527 2020-07-06  3601599666106972342   685813001  0.016932   
28675528 2020-07-06  3601599666106972342   685813001  0.016932   
28675529 2020-07-06  3601599666106972342   685813001  0.016932   
28675530 2020-07-06  3601599666106972342   685813001  0.016932   

          sales_channel_id  season  year  month  week time_diff  
28675451                 2       2   -28      7    93       NaT  
28675452                 2       2   -28      7    93    0 days  
28675453 

In [11]:
import pandas as pd

# Merge transactions and customers dataframes on 'customer_id'
merged_data = transactions.merge(customers, on='customer_id', how='inner')

# Merge the merged_data with the articles dataframe on 'article_id'
merged_data = merged_data.merge(articles, on='article_id', how='inner')

# Now you have a dataframe that contains all the relevant information

# Filter for rows where the same customer bought the same article
duplicate_sales = merged_data[merged_data.duplicated(subset=['customer_id', 'article_id'], keep=False)]

# You can now access the dates of these transactions
duplicate_sales_dates = duplicate_sales[['customer_id', 'article_id', 't_dat']]

print(duplicate_sales_dates)

                   customer_id  article_id      t_dat
6             8979857957419452   715624010 2019-09-05
7             8979857957419452   715624010 2019-09-05
15           32772099130663382   715624010 2020-01-19
16           32772099130663382   715624010 2020-01-22
19           39172497731037957   715624010 2019-08-13
...                        ...         ...        ...
31788203  17969406184413867807   487750043 2019-04-09
31788231  18079877935957048209   553212001 2018-12-09
31788232  18079877935957048209   553212001 2018-12-09
31788292  18297284838138651866   790940001 2019-08-21
31788293  18297284838138651866   790940001 2019-08-21

[7997298 rows x 3 columns]


In [12]:
import pandas as pd

# Merge transactions and customers dataframes on 'customer_id'
merged_data = transactions.merge(customers, on='customer_id', how='inner')

# Merge the merged_data with the articles dataframe on 'article_id'
merged_data = merged_data.merge(articles, on='article_id', how='inner')

# Now you have a dataframe that contains all the relevant information

# Group by customer_id and article_id and filter for items that are repurchased on different dates
duplicate_sales = merged_data.groupby(['customer_id', 'article_id'])['t_dat'].nunique().reset_index()
duplicate_sales = duplicate_sales[duplicate_sales['t_dat'] > 1]

print(duplicate_sales)


                   customer_id  article_id  t_dat
11              23962613628581   732842001      2
40              28847241659200   672598002      2
64              28847241659200   762846001      2
87              28847241659200   859076001      2
164             77117344919861   539723003      2
...                        ...         ...    ...
27306369  18446723086055369602   559139004      2
27306386  18446737527580148316   596400007      2
27306387  18446737527580148316   608776003      2
27306399  18446737527580148316   678942039      2
27306431  18446737527580148316   827968002      2

[1141216 rows x 3 columns]


In [13]:
# Calculate the number of unique articles and customers in duplicate_sales
unique_articles = duplicate_sales['article_id'].nunique()
unique_customers = duplicate_sales['customer_id'].nunique()

print(f"Number of unique articles in duplicate_sales: {unique_articles}")
print(f"Number of unique customers in duplicate_sales: {unique_customers}")

Number of unique articles in duplicate_sales: 63079
Number of unique customers in duplicate_sales: 377478
